# MLCQ Dataset

In [2]:
import numpy as np 
import pandas as pd
import os

In [3]:
os.listdir()

['mlcq.xlsx', 'mlcq.csv', '.git', '.vscode', '아카이브', 'data.py', 'data.ipynb']

In [87]:
mlcq = pd.read_csv('mlcq.csv')
mlcq.shape

(14739, 15)

In [88]:
mlcq[mlcq.sample_id==3698323]

,id,reviewer_id,sample_id,smell,severity,review_timestamp,type,code_name,repository,commit_hash,path,start_line,end_line,link,is_from_industry_relevant_project
1363,1901,5,3698323,blob,none,2019-03-29 14:32:15.701013,class,com.amazon.ask.dispatcher.request.handler.impl...,git@github.com:alexa/alexa-skills-kit-sdk-for-...,bf1e9ccc50d1f3f8408f887f70197ee288fd4bd9,/ask-sdk-core/src/com/amazon/ask/dispatcher/re...,26,59,https://github.com/alexa/alexa-skills-kit-sdk-...,1.0
1364,1902,5,3698323,data class,none,2019-03-29 14:32:15.703950,class,com.amazon.ask.dispatcher.request.handler.impl...,git@github.com:alexa/alexa-skills-kit-sdk-for-...,bf1e9ccc50d1f3f8408f887f70197ee288fd4bd9,/ask-sdk-core/src/com/amazon/ask/dispatcher/re...,26,59,https://github.com/alexa/alexa-skills-kit-sdk-...,1.0


In [27]:
mlcq = mlcq.drop_duplicates("sample_id")
mlcq.shape

(4770, 15)

In [34]:
len(mlcq[mlcq.severity != "none"]) 

702

MLCQ 유니크 샘플 4770개 중 스멜의 비율 : 14.7% (702개)
## sample_id 별 severity들을 하나로(severities) 합치기

In [106]:
def sevrity_to_numeric(severity):
    if severity == 'none':
        return 0
    elif severity == 'minor':
        return 1
    elif severity == 'major':
        return 2
    elif severity == 'critical':
        return 3
    else:
        print(f'sev: {severity}')
        raise Exception 

In [107]:
def combine_severities(df, sample):
    rows = df[df.sample_id == sample]
    return [i for i in rows.severity.array]

In [111]:
mlcq = pd.read_csv('mlcq.csv').sort_values(by="sample_id")
samples = mlcq.drop_duplicates("sample_id").sample_id
# severity 제외 공통된 부분만 픽
mlcq = mlcq.loc[:,["sample_id", "severity", "code_name", "repository", "commit_hash", "path", "start_line", "end_line", "link"]]
mlcq.severity = mlcq.severity.map(lambda s: sevrity_to_numeric(s))
severities = samples.map(lambda sample: combine_severities(mlcq, sample))

mlcq["severities"] = pd.Series(severities)
mlcq = mlcq.drop(["severity"], axis=1)
mlcq = mlcq.drop_duplicates(["sample_id"])
mlcq["avg"] = mlcq.severities.map(lambda ss: np.mean(ss))
mlcq

,sample_id,code_name,repository,commit_hash,path,start_line,end_line,link,severities,avg
1364,3698323,com.amazon.ask.dispatcher.request.handler.impl...,git@github.com:alexa/alexa-skills-kit-sdk-for-...,bf1e9ccc50d1f3f8408f887f70197ee288fd4bd9,/ask-sdk-core/src/com/amazon/ask/dispatcher/re...,26,59,https://github.com/alexa/alexa-skills-kit-sdk-...,"[0, 0]",0.000000
1357,3698602,com.amazon.ask.request.mapper.impl.BaseRequest...,git@github.com:alexa/alexa-skills-kit-sdk-for-...,bf1e9ccc50d1f3f8408f887f70197ee288fd4bd9,/ask-sdk-runtime/src/com/amazon/ask/request/ma...,79,81,https://github.com/alexa/alexa-skills-kit-sdk-...,"[0, 0]",0.000000
1359,3698665,com.amazon.ask.builder.impl.AbstractSkillBuild...,git@github.com:alexa/alexa-skills-kit-sdk-for-...,bf1e9ccc50d1f3f8408f887f70197ee288fd4bd9,/ask-sdk-runtime/src/com/amazon/ask/builder/im...,91,94,https://github.com/alexa/alexa-skills-kit-sdk-...,"[0, 0]",0.000000
1355,3698860,com.alibaba.android.arouter.facade.model.Route...,git@github.com:alibaba/ARouter.git,93b328569bbdbf75e4aa87f0ecf48c69600591b2,/arouter-annotation/src/main/java/com/alibaba/...,200,213,https://github.com/alibaba/ARouter/blob/93b328...,"[0, 0]",0.000000
1362,3699227,com.alibaba.android.arouter.facade.enums.#getC...,git@github.com:alibaba/ARouter.git,93b328569bbdbf75e4aa87f0ecf48c69600591b2,/arouter-annotation/src/main/java/com/alibaba/...,32,34,https://github.com/alibaba/ARouter/blob/93b328...,"[0, 0]",0.000000
...,...,...,...,...,...,...,...,...,...,...
7110,9540354,org.apache.hadoop.yarn.server.nodemanager.cont...,git@github.com:apache/hadoop.git,128dd91e10080bdcbcd7d555fa3c4105e55a6b51,/hadoop-yarn-project/hadoop-yarn/hadoop-yarn-s...,24,39,https://github.com/apache/hadoop/blob/128dd91e...,"[0, 0]",0.000000
10206,9542671,org.apache.hadoop.yarn.webapp.hamlet2.HamletSp...,git@github.com:apache/hadoop.git,128dd91e10080bdcbcd7d555fa3c4105e55a6b51,/hadoop-yarn-project/hadoop-yarn/hadoop-yarn-c...,1772,1820,https://github.com/apache/hadoop/blob/128dd91e...,"[0, 0, 0, 2, 0, 0]",0.333333
4781,9542718,org.apache.hadoop.mapred.NotRunningJob#killTas...,git@github.com:apache/hadoop.git,128dd91e10080bdcbcd7d555fa3c4105e55a6b51,/hadoop-mapreduce-project/hadoop-mapreduce-cli...,213,219,https://github.com/apache/hadoop/blob/128dd91e...,"[0, 0]",0.000000
9135,9548458,org.apache.hadoop.metrics2.sink.ganglia.Gangli...,git@github.com:apache/hadoop.git,128dd91e10080bdcbcd7d555fa3c4105e55a6b51,/hadoop-common-project/hadoop-common/src/main/...,46,253,https://github.com/apache/hadoop/blob/128dd91e...,"[0, 0]",0.000000
